In [1]:
from src.SensitivityAnalysis import load_data
import matplotlib.pyplot as plt
import pandas as pd
from src.SugarScape import SugarScape
from mesa import batch_run
import numpy as np
from tqdm import tqdm

# Finding Parameters

In [ ]:
data = load_data()
averaged = []
for i in range(len(data) // 10 - 1):
    trader_mean = data['Trader Count'].iloc[i*10:(i+1)*10].mean()
    gini_mean = data['Gini'].iloc[i*10:(i+1)*10].mean()
    
    if trader_mean > 0:
        averaged.append(1 / gini_mean * trader_mean)


In [ ]:
# Find where gini maximized 
worst = dict(data.iloc[np.argmin(averaged)])
best = dict(data.iloc[np.argmax(averaged)])

worst = {key: float(value) for (key,value) in worst.items()}
best = {key: float(value) for (key,value) in best.items()}

print(f'Worst case: {worst}')
print(f'Best case: {best}')

# Find closest to average
dist = (averaged - np.mean(averaged))**2
idx = np.argmin(dist)

typical = dict(data.iloc[idx * 10])
typical = {key: float(value) for (key,value) in typical.items()}
print(f'Typical: {typical}')


# Run Base Simulations

In [ ]:
## USe these map schemes based on your name
# Priyank: top-heavy
# Amish: uniform
# Ilia: split

map_scheme = 'split'

scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052, 'map_scheme': map_scheme},   # Worst case
    
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329, 'map_scheme': map_scheme},     # Best case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},  # Average case
    
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3, 'map_scheme': map_scheme},                                   # Balanced case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},                    # Slow evolving
    
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239, 'map_scheme': map_scheme},                    # Fast evolving
]

results = []
replicates = 30
max_steps = 200

In [ ]:
for scenario in scenarios:
    batch = batch_run(SugarScape, scenario, number_processes=None,
                      iterations=replicates, max_steps=max_steps, display_progress=True, data_collection_period=1)
    for i in range(replicates):
        scenario["Gini"] = [val['Gini'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
        scenario["Trader Count"] = [val['Trader Count'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]

        results.append(list(scenario.values()))

# Convert to df and save
names = list(scenario.keys()) 
df = pd.DataFrame(results, columns=names)
df.to_csv(f'Experiments/Base/results_{map_scheme}.csv')

In [ ]:
def safe_eval(value):
    try:
        return eval(value)
    except:
        return value

names = ['Worst', 'Best', 'Typical', 'Balanced', 'Slow evolving', 'Fast evolving']
for map_scheme in ['uniform', 'top_heavy', 'split']:
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    df = pd.read_csv(f'Experiments/Base/results_{map_scheme}.csv')
    df['Gini'] = df['Gini'].apply(safe_eval)
    df['Trader Count'] = df['Trader Count'].apply(safe_eval)
    
    for i in range(len(scenarios)):
        # Get gini and trader count of every scenario
        gini = df.iloc[i*replicates:(i+1)*replicates]['Gini']
        trader = df.iloc[i*replicates:(i+1)*replicates]['Trader Count']
        
        # Convert to 2d np arrays
        gini = np.vstack(gini)
        trader = np.vstack(trader)
        
        # Get means and ci
        gini_mean = np.mean(gini, axis=0)
        gini_ci = np.std(gini, ddof=1, axis=0) / replicates * 1.96
        trader_mean = np.mean(trader, axis=0)
        trader_ci = np.std(trader, ddof=1, axis=0) / replicates * 1.96
        
        # Plotting
        x = np.arange(len(gini_mean))
        ax[0].plot(x, gini_mean, label=names[i])
        ax[0].fill_between(x, gini_mean + gini_ci, gini_mean - gini_ci, alpha=0.5)
        ax[1].plot(x, trader_mean, label=names[i])
        ax[1].fill_between(x, trader_mean + trader_ci, trader_mean - trader_ci, alpha=0.5)
    
    # Set labels, grid and title
    ax[1].legend(bbox_to_anchor=(1, 1.05))
    ax[0].set_title('Gini')
    ax[1].set_title('Trader Count')
    for i in range(2):
        ax[i].grid()
        ax[i].set_xlabel('Timesteps')
    ax[0].set_ylabel('Gini Coefficient')
    ax[1].set_ylabel('Number of Traders')
    
    # Save figure
    fig.suptitle(f'{map_scheme}')
    fig.tight_layout()
    fig.savefig(f'Experiments/Base/plot_{map_scheme}.png', dpi=300, bbox_inches='tight')

# Running Experiments

In [ ]:
## Use these map schemes based on your name
# Priyank: top-heavy
# Amish: uniform
# Ilia: split

map_scheme = 'uniform'

scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052, 'map_scheme': map_scheme},   # Worst case
    
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329, 'map_scheme': map_scheme},     # Best case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},  # Average case
    
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3, 'map_scheme': map_scheme},                                   # Balanced case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},                    # Slow evolving
    
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239, 'map_scheme': map_scheme},                    # Fast evolving
]


tax_systems = [
    ("progressive", "needs"),
    ("flat", "flat"),
    ("regressive", "random"),
    ("luxury", "progressive"),
    ("progressive", "progressive")
]

tax_rates = [0.1, 0.25, 0.4]

results = []
replicates = 30
max_steps = 200

In [ ]:
with tqdm(total=len(scenarios), ncols=70) as pbar:
    for scenario in scenarios:
        for tax_system in tax_systems:
            for tax_rate in tax_rates:
                scenario_copy = scenario.copy()
                # Adding taxsystem and rate into scenario
                scenario_copy['tax_scheme'] = tax_system[0]
                scenario_copy['distributer_scheme'] = tax_system[1]
                scenario_copy['tax_rate'] = tax_rate
        
                batch = batch_run(SugarScape, scenario_copy, number_processes=None,
                                  iterations=replicates, max_steps=max_steps, display_progress=False, data_collection_period=1)
                for i in range(replicates):
                    scenario_copy["Gini"] = [val['Gini'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                    scenario_copy["Trader Count"] = [val['Trader Count'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
    
                results.append(list(scenario_copy.values()))
    
        pbar.update(1)

# Convert to df and save
names = list(scenario_copy.keys()) 
df = pd.DataFrame(results, columns=names)
df.to_csv(f'Experiments/results_{map_scheme}.csv')

# Updated Final Experiment

In [2]:
import logging
from multiprocessing import Pool, cpu_count
# from src.SugarScape import SugarScape, batch_run

def setup_logger():
    """Set up a logger for simulation."""
    logger = logging.getLogger("simulation")
    handler = logging.FileHandler("simulation.log")
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    return logger

map_scheme = 'top_heavy'

scenarios = [
    {'vision_mean': 4.264199728146195, 'metabolism_mean': 6.751366691663861, 'max_age_mean': 96.55008263885976,
     'repopulate_factor': 7.485186895355582, 'cell_regeneration': 1.6743798777461052, 'map_scheme': map_scheme},   # Worst case
    
    {'vision_mean': 2.571960593573749, 'metabolism_mean': 7.285224918276071, 'max_age_mean': 93.18476642481984,
     'repopulate_factor': 5.733334645628929, 'cell_regeneration': 2.95167014375329, 'map_scheme': map_scheme},     # Best case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 96.730439318344,
     'repopulate_factor': 12.009924734011292, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},  # Average case
    
    {'vision_mean': 3.5, 'metabolism_mean': 6, 'max_age_mean': 85,
     'repopulate_factor': 10, 'cell_regeneration': 3, 'map_scheme': map_scheme},                                   # Balanced case
    
    {'vision_mean': 1.2129357354715466, 'metabolism_mean': 3.5813456028699875, 'max_age_mean': 100,
     'repopulate_factor': 5, 'cell_regeneration': 1.5217601098120213, 'map_scheme': map_scheme},                    # Slow evolving
    
    {'vision_mean': 5.089661035686731, 'metabolism_mean': 5.488567331805825, 'max_age_mean': 70,
     'repopulate_factor': 15, 'cell_regeneration': 3.368525765836239, 'map_scheme': map_scheme},                    # Fast evolving
]

tax_systems = [
    ("progressive", "needs"),
    ("flat", "flat"),
    ("regressive", "random"),
    ("luxury", "progressive"),
    ("progressive", "progressive")
]

tax_rates = [0.1, 0.25, 0.4]

results = []
replicates = 10
max_steps = 200

def run_experiment(scenario):
    """Run the SugarScape model with given parameters and return results."""
    logger = setup_logger()
    results = []
    for tax_system in tax_systems:
        for tax_rate in tax_rates:
            scenario_copy = scenario.copy()
            scenario_copy['tax_scheme'] = tax_system[0]
            scenario_copy['distributer_scheme'] = tax_system[1]
            scenario_copy['tax_rate'] = tax_rate
            
            logger.info(f"Running experiment: {scenario_copy}")
            batch = batch_run(SugarScape, scenario_copy, number_processes=None,
                              iterations=replicates, max_steps=max_steps, display_progress=False, data_collection_period=1)
            for i in range(replicates):
                scenario_result = scenario_copy.copy()
                scenario_result["Gini"] = [val['Gini'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                scenario_result["Trader Count"] = [val['Trader Count'] for val in batch[(max_steps+1)*i:(max_steps+1)*(i+1)]]
                results.append(list(scenario_result.values()))
    return results

with tqdm(total=len(scenarios), ncols=70) as pbar:
    for scenario in scenarios:
        results.extend(run_experiment(scenario))
        pbar.update(1)

# Convert to df and save
names = list(scenario.keys()) + ['tax_scheme', 'distributer_scheme', 'tax_rate', 'Gini', 'Trader Count']
df = pd.DataFrame(results, columns=names)
df.to_csv(f'Experiments/results_v4_{map_scheme}.csv', index=False)


100%|██████████████████████████████████| 6/6 [21:16<00:00, 212.77s/it]
